In [1]:
#r "nuget:Microsoft.ML.FastTree,1.4.0-preview2"
#r "nuget:Microsoft.ML.DotNet.Interactive.Extensions"

using Microsoft.ML.DotNet.Interactive;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.FastTree;

Installing package Microsoft.ML.FastTree, version 1.4.0-preview2............done!

Successfully added reference to package Microsoft.ML.FastTree, version 1.4.0-preview2

Installing package Microsoft.ML.DotNet.Interactive.Extensions....done!

Successfully added reference to package Microsoft.ML.DotNet.Interactive.Extensions, version 1.0.0

Loading kernel extensions in directory /home/jovyan/.nuget/packages/microsoft.ml.dotnet.interactive.extensions/1.0.0/interactive-extensions/dotnet/cs/

Loading kernel extension Microsoft.ML.DotNet.Interactive.MlKernelExtension from assembly /home/jovyan/.nuget/packages/microsoft.ml.dotnet.interactive.extensions/1.0.0/interactive-extensions/dotnet/cs/Microsoft.ML.DotNet.Interactive.Extensions.dll

Loaded kernel extension Microsoft.ML.DotNet.Interactive.MlKernelExtension from assembly /home/jovyan/.nuget/packages/microsoft.ml.dotnet.interactive.extensions/1.0.0/interactive-extensions/dotnet/cs/Microsoft.ML.DotNet.Interactive.Extensions.dll

Loaded kernel extensions in directory /home/jovyan/.nuget/packages/microsoft.ml.dotnet.interactive.extensions/1.0.0/interactive-extensions/dotnet/cs/

# Load Model
Load the model from disk. Get the FastTree transformer from it. And get the names of each feature column.

In [2]:
MLContext mlContext = new MLContext();
ITransformer model = mlContext.Model.Load("HousingTreeModel.zip", out DataViewSchema inputSchema);

DataViewSchema outputSchema = model.GetOutputSchema(inputSchema);

ISingleFeaturePredictionTransformer<TreeEnsembleModelParametersBasedOnRegressionTree> predictor = ((TransformerChain<ITransformer>)model).LastTransformer as ISingleFeaturePredictionTransformer<TreeEnsembleModelParametersBasedOnRegressionTree>;
VBuffer<ReadOnlyMemory<char>> featureNames = default;
outputSchema[predictor.FeatureColumnName].GetSlotNames(ref featureNames);

# Display the Tree

In [ ]:
predictor.Model.TrainedTreeEnsemble.ToDecisionTreeData(featureNames)